In [ ]:
# Esse robo mantem os arquivos referentes a carteira atualizado nas pastas para os usuários fazerem consultas

import pandas as pd
import pyodbc
from openpyxl import Workbook, load_workbook
import time
import win32com.client as win32
from datetime import datetime

def conexao():
    server = '172.20.1.35\PRODUCAO'
    database = 'CorporeRM'
    username = ''
    password = ''
    driver= ''
    con = 'Yes'
    conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password+'; MARS_Connection='+con)
            # Buscando o cursor e apontando para a conexão.
    cursor = conn.cursor()
    query_rec = """
                -- Sem Serviço Extra

                select 
                    CTR_NRO as 'numero_contrato',
                    CTR_DTA as 'data_contrato',
                    CPR_DTA_INI as 'data_inicial_item',
                    CPR_DTA_INI_FAT as 'data_inicial_faturamento_item',
                    CPR_DTA_FIN_FAT as 'data_final_faturamento_item',
                    cast(contrato as money) as 'valor_contrato',
                    cast(produto as money) as 'valor_item',
                    fct_dsc as 'status_contrato',
                    prd_nom as 'descricao_item',
                    servico_extra,
                    cast(case	when day(CPR_DTA_INI_FAT) = 31 and CPR_DTA_FIN_FAT is null then 0 
                                when fct_dsc = 'Em Vigência' and year(CPR_DTA_INI_FAT) = year(getdate()) and month(CPR_DTA_INI_FAT) = month(getdate()) and CPR_DTA_FIN_FAT is null then
                                    cast(round(produto * (30 - day(CPR_DTA_INI_FAT)) / 30, 2) as money)
                                when fct_dsc = 'Em Vigência' and year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) and day(CPR_DTA_FIN_FAT) - day(CPR_DTA_INI_FAT) <> 0 then
                                    cast(round(produto * day(CPR_DTA_FIN_FAT) / 30, 2) as money)
                                when fct_dsc = 'Encerrado' and year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) then 
                                    cast(round(produto * (30 - day(CPR_DTA_FIN_FAT)) / 30, 2) as money)
                                when year(CPR_DTA_INI_FAT) > year(getdate()) or
                                    year(CPR_DTA_INI_FAT) = year(getdate()) and month(CPR_DTA_INI_FAT) > month(getdate()) then 0
                                when fct_dsc = 'Encerrado' and CPR_DTA_FIN_FAT is null then produto 
                                when fct_dsc = 'Em Vigência' then produto 
                    end as money) as 'valor_total_contrato_recalculado',
                    format(getdate(), 'dd/MM/yyyy') as 'data_dados'
                from (select * from (select  
                                        CTR_NRO,
                                        CTR_DTA,
                                        RAI_DSC,
                                        FIL_DSC,
                                        EMP_IDE,
                                        EMP_APE,
                                        EMP_NOM_COM,
                                        OGA_NOT,
                                        OGA_DIU,
                                        FCT_DSC,
                                        cast(CTR_VAL_TOT as money) as 'contrato',
                                        cast(CPR_VAL_TOT as money) as 'produto',
                                        LTR_DSC,
                                        PRD_NOM,
                                        CPR_DTA_INI,
                                        CPR_DTA_INI_FAT,
                                        CPR_DTA_FIN_FAT,
                                        CPR_FLG_ENC,
                                        case when CPR_FLG_EXT = 1 or CME_IDE = 17 or PRD_NOM like('%Serviço Extra%') then 'Sim' else 'Não' end as 'servico_extra'
                                    FROM [GR2DB\PRODUCAO].[SM].dbo.v_com_contratos_detalhado
                                    where 
                                        CPR_FLG_ENC = 0 and -- 0 é porque está ativo  and--and ctr_nro = '255/2022' and
                                        FCT_DSC <> 'Elaboração' and --and
                                        case when CPR_DTA_fin_FAT is null then 'ok' 
                                                when year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) then 'ok'
                                                else 'não ok' end = 'ok') as t1
                        where 
                            servico_extra = 'Não'
                        union 
                        select * from (select  
                                        CTR_NRO,
                                        CTR_DTA,
                                        RAI_DSC,
                                        FIL_DSC,
                                        EMP_IDE,
                                        EMP_APE,
                                        EMP_NOM_COM,
                                        OGA_NOT,
                                        OGA_DIU,
                                        FCT_DSC,
                                        cast(CTR_VAL_TOT as money) as 'contrato',
                                        cast(CPR_VAL_TOT as money) as 'produto',
                                        LTR_DSC,
                                        PRD_NOM,
                                        CPR_DTA_INI,
                                        CPR_DTA_INI_FAT,
                                        CPR_DTA_FIN_FAT,
                                        CPR_FLG_ENC,
                                        case when CPR_FLG_EXT = 1 or CME_IDE = 17 or PRD_NOM like('%Serviço Extra%') then 'Sim' else 'Não' end as 'servico_extra'
                                    FROM [GR2DB\PRODUCAO].[SM].dbo.v_com_contratos_detalhado
                                    where 
                                        CPR_FLG_ENC = 1 and -- 1 É porque está encerrado
                                        FCT_DSC <> 'Elaboração' and 
                                        year(CPR_DTA_FIN) = year(getdate()) and
                                        month(CPR_DTA_FIN) = month(getdate())) as t2
                        where 
                            servico_extra = 'Não' 
                    ) as t3
                order by
                        CPR_DTA_INI
            """
    
    query_extra = """
                select 
                    CTR_NRO as 'numero_contrato',
                    CTR_DTA as 'data_contrato',
                    CPR_DTA_INI as 'data_inicial_item',
                    CPR_DTA_INI_FAT as 'data_inicial_faturamento_item',
                    CPR_DTA_FIN_FAT as 'data_final_faturamento_item',
                    cast(contrato as money) as 'valor_contrato',
                    cast(produto as money) as 'valor_item',
                    fct_dsc as 'status_contrato',
                    prd_nom as 'descricao_item',
                    servico_extra,
                    cast( case	when day(CPR_DTA_INI_FAT) = 31 and CPR_DTA_FIN_FAT is null then 0
                                when fct_dsc = 'Em Vigência' and year(CPR_DTA_INI_FAT) = year(getdate()) and month(CPR_DTA_INI_FAT) = month(getdate()) and CPR_DTA_FIN_FAT is null then
                                    cast(round(produto * (30 - day(CPR_DTA_INI_FAT)) / 30, 2) as money)
                                when fct_dsc = 'Em Vigência' and year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) and day(CPR_DTA_FIN_FAT) - day(CPR_DTA_INI_FAT) <> 0 then
                                    cast(round(produto * day(CPR_DTA_FIN_FAT) / 30, 2) as money)
                                when fct_dsc = 'Encerrado' and year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) then 
                                    cast(round(produto * (30 - day(CPR_DTA_FIN_FAT)) / 30, 2) as money)
                                when year(CPR_DTA_INI_FAT) > year(getdate()) or
                                    year(CPR_DTA_INI_FAT) = year(getdate()) and month(CPR_DTA_INI_FAT) > month(getdate()) then 0
                                when fct_dsc = 'Encerrado' and CPR_DTA_FIN_FAT is null then produto 
                                when fct_dsc = 'Em Vigência' then produto 
                          end as money) as 'valor_total_contrato_recalculado'
                from (select * from (select  
                                        CTR_NRO,
                                        CTR_DTA,
                                        RAI_DSC,
                                        FIL_DSC,
                                        EMP_IDE,
                                        EMP_APE,
                                        EMP_NOM_COM,
                                        OGA_NOT,
                                        OGA_DIU,
                                        FCT_DSC,
                                        cast(CTR_VAL_TOT as money) as 'contrato',
                                        cast(CPR_VAL_TOT as money) as 'produto',
                                        LTR_DSC,
                                        PRD_NOM,
                                        CPR_DTA_INI,
                                        CPR_DTA_INI_FAT,
                                        CPR_DTA_FIN_FAT,
                                        case when CPR_FLG_EXT = 1 or CME_IDE = 17 or PRD_NOM like('%Serviço Extra%') then 'Sim' else 'Não' end as 'servico_extra'
                                    FROM [GR2DB\PRODUCAO].[SM].dbo.v_com_contratos_detalhado
                                    where 
                                        CPR_FLG_ENC = 0 and -- 0 é porque está ativo  and--and ctr_nro = '255/2022' and
                                        FCT_DSC <> 'Elaboração' and --and
                                        case when CPR_DTA_fin_FAT is null then 'ok' 
                                                when year(CPR_DTA_FIN_FAT) = year(getdate()) and month(CPR_DTA_FIN_FAT) = month(getdate()) then 'ok'
                                                else 'não ok' end = 'ok') as t1
                        where 
                            servico_extra = 'Sim'
                        union 
                        select * from (select  
                                        CTR_NRO,
                                        CTR_DTA,
                                        RAI_DSC,
                                        FIL_DSC,
                                        EMP_IDE,
                                        EMP_APE,
                                        EMP_NOM_COM,
                                        OGA_NOT,
                                        OGA_DIU,
                                        FCT_DSC,
                                        cast(CTR_VAL_TOT as money) as 'contrato',
                                        cast(CPR_VAL_TOT as money) as 'produto',
                                        LTR_DSC,
                                        PRD_NOM,
                                        CPR_DTA_INI,
                                        CPR_DTA_INI_FAT,
                                        CPR_DTA_FIN_FAT,
                                        case when CPR_FLG_EXT = 1 or CME_IDE = 17 or PRD_NOM like('%Serviço Extra%') then 'Sim' else 'Não' end as 'servico_extra'
                                    FROM [GR2DB\PRODUCAO].[SM].dbo.v_com_contratos_detalhado
                                    where 
                                        CPR_FLG_ENC = 1 and -- 1 É porque está encerrado
                                        FCT_DSC <> 'Elaboração' and 
                                        year(CPR_DTA_FIN) = year(getdate()) and
                                        month(CPR_DTA_FIN) = month(getdate())) as t2
                        where 
                            servico_extra = 'Sim' 
                    ) as t3
    
    """
    
    carteira_recorrente = pd.read_sql(query_rec, conn)
    carteira_extra = pd.read_sql(query_extra, conn)
    conn.commit()
    return carteira_recorrente, carteira_extra

carteira_recorrente, carteira_extra = conexao()

df_carteira_recorrente = pd.DataFrame(data=carteira_recorrente)

df_carteira_extra = pd.DataFrame(data=carteira_extra)

month = str(datetime.now().strftime('%m')) + str(datetime.now().strftime('%Y'))

caminho_arq = f'G:/Drives compartilhados/FATURAMENTO/carteira_tt/carteira_{month}.xlsx'

with pd.ExcelWriter(caminho_arq) as writer:
    df_carteira_recorrente.to_excel(writer, sheet_name = 'Recorrente', index=False)
    df_carteira_extra.to_excel(writer, sheet_name = 'Extra', index=False)

C:\Users\daniel.tavares\AppData\Local\Temp\ipykernel_17932\372272193.py:192: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  carteira_recorrente = pd.read_sql(query_rec, conn)
C:\Users\daniel.tavares\AppData\Local\Temp\ipykernel_17932\372272193.py:193: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  carteira_extra = pd.read_sql(query_extra, conn)
